Importing transfermarkt data from Data World - David Cereijo

In [31]:
import pandas as pd
import os

repo_dir = os.getcwd()  # Directory of the script
transferMKT_dir = os.path.join(repo_dir, 'transferMKT-data')

In [37]:
# Global Variables:
my_leagues = ['ES1', 'IT1', 'GR1', 'GB1', 'FR1']
player_cols = ['date_y', 'player_id', 'player_name', 'player_current_club_id', 'market_value_in_eur', 'player_club_domestic_competition_id']

In [32]:
# Function that creates a CSV from a pandas df:
def make_csv(df, dir, file_name):
    file_path = os.path.join(dir, f'{file_name}.csv')
    return df.to_csv(file_path, index=True)

In [33]:
# Create a pandas df for the appearances file:
appearances_df = pd.read_csv('https://query.data.world/s/2t4a5mgcrt7xb32ifpci2wijahs7fq?dws=00000')

# Create a pandas dataframe:
player_val_df = pd.read_csv('https://query.data.world/s/bxh6i5g3kll34aqabzszjbecgdzabm?dws=00000')

In [34]:
# Merge the appearances and player_val df:
merge1 = pd.merge(appearances_df, player_val_df, on='player_id')

In [38]:
# Make a copy of merge 1 and slice it to include only the necessary columns:
merge2 = merge1.copy(deep=True)

merge3 = merge2[player_cols]
merge3

,date_y,player_id,player_name,player_current_club_id,market_value_in_eur,player_club_domestic_competition_id
0,2012-12-02,172127,Tyronne,3999,200000,GR1
1,2014-08-10,172127,Tyronne,3999,200000,GR1
2,2015-07-01,172127,Tyronne,3999,400000,GR1
3,2016-02-22,172127,Tyronne,3999,300000,GR1
4,2016-07-15,172127,Tyronne,3999,300000,GR1
...,...,...,...,...,...,...
27777120,2023-01-06,668606,Yuito Suzuki,667,700000,FR1
27777121,2020-07-15,762387,Khusrav Toirov,660,25000,UKR1
27777122,2020-10-28,762387,Khusrav Toirov,660,25000,UKR1
27777123,2022-11-25,762387,Khusrav Toirov,660,150000,UKR1


In [40]:
# Filter the merged dataframe on league ids and date:
filter_date = '2017-05-31'

filtered_df = merge3[merge3['player_club_domestic_competition_id'].isin(my_leagues) & (merge3['date_y'] >= filter_date)]

In [44]:
# Generate a list of player names for each league:
league_players_lists = [list(filtered_df[filtered_df['player_club_domestic_competition_id'] == league]['player_name'].unique()) for league in my_leagues]

# Assign each league to one of the lists:
laliga_players = league_players_lists[0]
serieA_players = league_players_lists[1]
bundesliga_players = league_players_lists[2]
epl_players = league_players_lists[3]
ligue1_players = league_players_lists[4]

In [45]:
# Create a dataframe for each league using a list comprehension:
league_player_dfs = [filtered_df[filtered_df['player_club_domestic_competition_id'] == league] for league in my_leagues]